In [1]:
%matplotlib inline
import nltk
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import inaugural, state_union, stopwords
import re
from sklearn.model_selection import train_test_split

In [2]:
nltk.download('inaugural')
nltk.download('state_union')

[nltk_data] Downloading package inaugural to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package inaugural is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!


True

In [3]:
print(state_union.fileids())

['1945-Truman.txt', '1946-Truman.txt', '1947-Truman.txt', '1948-Truman.txt', '1949-Truman.txt', '1950-Truman.txt', '1951-Truman.txt', '1953-Eisenhower.txt', '1954-Eisenhower.txt', '1955-Eisenhower.txt', '1956-Eisenhower.txt', '1957-Eisenhower.txt', '1958-Eisenhower.txt', '1959-Eisenhower.txt', '1960-Eisenhower.txt', '1961-Kennedy.txt', '1962-Kennedy.txt', '1963-Johnson.txt', '1963-Kennedy.txt', '1964-Johnson.txt', '1965-Johnson-1.txt', '1965-Johnson-2.txt', '1966-Johnson.txt', '1967-Johnson.txt', '1968-Johnson.txt', '1969-Johnson.txt', '1970-Nixon.txt', '1971-Nixon.txt', '1972-Nixon.txt', '1973-Nixon.txt', '1974-Nixon.txt', '1975-Ford.txt', '1976-Ford.txt', '1977-Ford.txt', '1978-Carter.txt', '1979-Carter.txt', '1980-Carter.txt', '1981-Reagan.txt', '1982-Reagan.txt', '1983-Reagan.txt', '1984-Reagan.txt', '1985-Reagan.txt', '1986-Reagan.txt', '1987-Reagan.txt', '1988-Reagan.txt', '1989-Bush.txt', '1990-Bush.txt', '1991-Bush-1.txt', '1991-Bush-2.txt', '1992-Bush.txt', '1993-Clinton.txt',

In [4]:
fdrfilenames = ['1933-Roosevelt.txt', '1937-Roosevelt.txt', '1941-Roosevelt.txt', '1945-Roosevelt.txt']
rrfilenames = ['1981-Reagan.txt', '1985-Reagan.txt']

In [5]:
with open("fdr.txt", "w") as f:
    for filename in fdrfilenames:
        f.write(nltk.corpus.inaugural.raw(filename))

In [6]:
with open("rr.txt", "w") as f:
    for filename in rrfilenames:
        f.write(nltk.corpus.inaugural.raw(filename))

In [7]:
with open('fdr.txt', 'r') as myfile:
    Roosevelt = myfile.read()
with open('rr.txt', 'r') as myfile:
    Reagan = myfile.read()

In [8]:
Reaganclean = re.sub("\n", "", Reagan)
Reaganclean = re.sub("--", " ", Reaganclean)
Reagan = Reaganclean

In [9]:
Rooseveltclean = re.sub("\n", "", Roosevelt)
Rooseveltclean = re.sub("--", " ", Rooseveltclean)
Roosevelt = Rooseveltclean

In [10]:
nlp = spacy.load('en')
Reagan_doc = nlp(Reagan)
Roosevelt_doc = nlp(Roosevelt)

In [12]:
Reagan_sents = [[sent, "Reagan"] for sent in Reagan_doc.sents]

In [13]:
Roosevelt_sents = [[sent, "Roosevelt"] for sent in Roosevelt_doc.sents]

In [14]:
sentences = pd.DataFrame(Reagan_sents + Roosevelt_sents)
sentences_tfidf = pd.DataFrame(Reagan_sents + Roosevelt_sents)

In [15]:
null_count1 = sentences_tfidf.isnull().sum()
null_count1[null_count1>0]

Series([], dtype: int64)

In [16]:
def most_used_words(text):
    allwords = [token.lemma_ for token in text
               if not token.is_punct and not token.is_stop]
    return [item[0] for item in Counter(allwords).most_common(100)]



In [17]:
def common_word_features(sentences, common_words):
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    for i, sentence in enumerate(df['text_sentence']):
        words = [token.lemma_
                for token in sentence
                if (
                not token.is_punct
                and not token.is_stop
                and token.lemma_ in common_words
                )]
        for word in words:
            df.loc[i, word] += 1
            
    return df



In [18]:
from collections import Counter
Reaganwords = most_used_words(Reagan_doc)
Rooseveltwords = most_used_words(Roosevelt_doc)

In [20]:
presidential_words = set(Reaganwords + Rooseveltwords)

In [21]:
word_counts = common_word_features(sentences, presidential_words)

In [22]:
word_counts.head()

,-PRON-,growth,effort,free,sacred,call,dream,action,tax,strength,...,civilization,hope,measure,hero,think,great,value,economic,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Senator, Hatfield, ,, Mr., Chief, Justice, ,,...",Reagan
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,"(The, orderly, transfer, of, authority, as, ca...",Reagan
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(In, the, eyes, of, many, in, the, world, ,, t...",Reagan
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Mr, .)",Reagan
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(President, ,, I, want, our, fellow, citizens,...",Reagan


In [23]:
len(presidential_words)

156

In [24]:
from sklearn.svm import SVC

# Instantiate our model and fit the data.
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.2,random_state=0)
svm = SVC(kernel = 'linear')
svm.fit(X_train, y_train)
print('Training set score:', svm.score(X_train, y_train))
print('\nTest set score:', svm.score(X_test, y_test))

Training set score: 0.8591885441527446

Test set score: 0.7047619047619048


In [25]:
svm_predicted = svm.predict(X_test)
pd.crosstab(y_test, svm_predicted)

col_0,Reagan,Roosevelt
text_source,,
Reagan,39,13
Roosevelt,18,35


In [26]:
#sentences_tfidf.rename(columns={0: "a", 1: "b"})
#sentences_tfidf[0] = sentences_tfidf[0].astype(int).astype('str')


In [27]:
'''
def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
sentences_tfidf['a'] = sentences_tfidf.a.apply(custom_tokenize)
'''

"\ndef custom_tokenize(text):\n    if not text:\n        print('The text to be tokenized is a None type. Defaulting to blank string.')\n        text = ''\n    return word_tokenize(text)\nsentences_tfidf['a'] = sentences_tfidf.a.apply(custom_tokenize)\n"

In [28]:
#nlp = spacy.load('en')
#emma = ' '
#emma = emma.join(emma_paras)
sentences_tfidf.loc[sentences_tfidf[1] == 'Reagan', 1] = 1
sentences_tfidf.loc[sentences_tfidf[1] == 'Roosevelt', 1] = 0

sentences_tfidf_X = sentences_tfidf[0]
sentences_tfidf_y = sentences_tfidf[1]
##emma = [[sent, 'emma']for sent in emma.sents]

In [29]:
sentences_tfidf_y = sentences_tfidf_y.astype(int)

In [30]:
str(sentences_tfidf_X[0])

"Senator Hatfield, Mr. Chief Justice, Mr. President, Vice President Bush, Vice President Mondale, Senator Baker, Speaker O'Neill, Reverend Moomaw, and my fellow citizens: To a few of us here today, this is a solemn and most momentous occasion; and yet, in the history of our Nation, it is a commonplace occurrence."

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [32]:
#X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(sentences_tfidf_X, sentences_tfidf_y, test_size=0.4, random_state=0)


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer(max_df=0.5, # drop words that occur in more than half the paragraphs
                             min_df=2, # only use words that appear at least twice
                             stop_words='english', 
                             lowercase=False, #convert everything to lower case (since Alice in Wonderland has the HABIT of CAPITALIZING WORDS for EMPHASIS)
                             use_idf=True,#we definitely want to use inverse document frequencies in our weighting
                             #norm=u'l2', #Applies a correction factor so that longer paragraphs and shorter paragraphs get treated equally
                             #smooth_idf=True #Adds 1 to all document frequencies, as if an extra document existed that used every word once.  Prevents divide-by-zero errors
                            )
#X_train_tfidf = X_train_tfidf.apply(lambda x:x.text)
#X_test_tfidf = X_test_tfidf.apply(lambda x:x.text)
sentences_tfidf_X = sentences_tfidf_X.apply(lambda x:x.text)

#Applying the vectorizer
sentences_tfidf_X = vectorizer.fit_transform(sentences_tfidf_X)
#X_train_tfidf=vectorizer.fit_transform(X_train_tfidf)
#X_test_tfidf=vectorizer.fit_transform(X_test_tfidf)

#print("Number of features: %d" % sentences_tfidf.get_shape()[1])

#splitting into training and test sets
#X_train_tfidf, X_test_tfidf= train_test_split(sentences_tfidf, test_size=0.4, random_state=0)



In [34]:
#TfidfVectorizer.fit_transform?

In [35]:
sentences_tfidf_y

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     1
11     1
12     1
13     1
14     1
15     1
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
      ..
494    0
495    0
496    0
497    0
498    0
499    0
500    0
501    0
502    0
503    0
504    0
505    0
506    0
507    0
508    0
509    0
510    0
511    0
512    0
513    0
514    0
515    0
516    0
517    0
518    0
519    0
520    0
521    0
522    0
523    0
Name: 1, Length: 524, dtype: int32

In [40]:
#vectorizer.get_feature_names()

In [ ]:
#y_train_tfidf = y_train_tfidf.astype('int')

In [39]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(sentences_tfidf_X, sentences_tfidf_y, test_size=0.4, random_state=0)


In [41]:
from sklearn.svm import SVC

# Instantiate our model and fit the data.
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

#X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_tfidf_test_tfidf_split(X, Y,test_tfidf_size=0.2,random_state=0)
svm = SVC(kernel = 'linear')
svm.fit(X_train_tfidf, y_train_tfidf)
print('train_tfidfing set score:', svm.score(X_train_tfidf, y_train_tfidf))
print('\ntest_tfidf set score:', svm.score(X_test_tfidf, y_test_tfidf))

train_tfidfing set score: 0.9745222929936306

test_tfidf set score: 0.7


In [43]:
svm_predicted = svm.predict(X_test_tfidf)
pd.crosstab(y_test_tfidf, svm_predicted)

col_0,0,1
1,,
0,82,22
1,41,65
